# Linear Regression

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
num_data = 1000
sample_data = []
for i in range(num_data):
    x1 = np.random.normal(0.0, 0.5)
    y1 = 0.1 * x1 + 0.3 + np.random.normal(0.0, 0.02)
    sample_data.append([x1, y1])
#print(sample_data)

In [ ]:
import matplotlib.pyplot as plt

x_data = [v[0] for v in sample_data]
y_data = [v[1] for v in sample_data]

plt.plot(x_data, y_data, 'ro', label='Sample data')
plt.legend()
plt.show()

In [ ]:
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))

print(W)
print(b)

In [ ]:
y = W * x_data + b

print(y)

In [ ]:
loss = tf.reduce_mean(tf.square(y - y_data))

print(loss)

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(0.5)


In [ ]:
train = optimizer.minimize(loss)

In [ ]:
%%time

sess = tf.Session()
sess.run(tf.global_variables_initializer())

num_epochs = 10

for step in range(num_epochs):
    sess.run(train)
    
    print(step, sess.run(W), sess.run(b), sess.run(loss))
    
    plt.plot(x_data, y_data, 'ro')
    y_ = W * x_data + b
    plt.plot(x_data, sess.run(y_), 'b')
    plt.plot(x_data, sess.run(W)*x_data+sess.run(b))
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim(-2, 2)
    plt.ylim(0.1, 0.6)
    plt.show()



# SOFTMAX REGRESSION (Image Classification)

## MNIST DATA SET

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_DATA/", one_hot=True)

X_train = mnist.train.images
Y_train = mnist.train.labels
X_test = mnist.test.images
Y_test = mnist.test.labels

print("X_train.shape: ", X_train.shape)
print("Y_train.shape: ", Y_train.shape)
print("X_test.shape: ", X_test.shape)
print("Y_test.shape: ", Y_test.shape)

digit_img = X_train[0]
digit_label = Y_train[0]
print("digit_img.shape: ", digit_img.shape)
print("digit_label.shape: ", digit_label.shape)
plt.imshow(digit_img.reshape(28, 28), cmap='gray')
plt.title(digit_label)
plt.show()


In [ ]:
plt.imshow(digit_img.reshape(28, 28))
plt.show()

print(digit_img.shape)
print(digit_img.reshape(28,28)[10,0:20])

# MNIST Image Classification using Softmax Regression

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

data = input_data.read_data_sets("./MNIST_DATA", one_hot=True)

In [ ]:
# parameters
NUM_STEPS = 1000
MINIBATCH_SIZE = 100

In [ ]:
# input placeholder
x = tf.placeholder(tf.float32, [None, 784])
y_true = tf.placeholder(tf.float32, [None, 10])

# variables
W = tf.Variable(tf.zeros([784, 10]))

# logit
y_pred = tf.matmul(x, W)

# loss
cross_entropy = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits(
                        logits=y_pred, labels=y_true))
# optimizer
gd_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# accuracy
correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))


In [ ]:
%%time
# train
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for _ in range(NUM_STEPS):
    batch_xs, batch_ys = data.train.next_batch(MINIBATCH_SIZE)
    sess.run(gd_step, feed_dict={x: batch_xs, 
                                 y_true: batch_ys})

In [ ]:
%%time
# test
ans = sess.run(accuracy, feed_dict={x: data.test.images,
                                    y_true: data.test.labels})
print("Accuracy: {:.4}%".format(ans*100))

In [ ]:
# predict
test_num = random.randint(0, data.test.images.shape[0])

img = mnist.test.images[test_num]
pred = sess.run(tf.argmax(y_pred, 1), feed_dict={x: [img]})
print("Predicted Label: ", pred[0])
print("Real Label: ", np.argmax(data.test.labels[test_num]))

img = np.resize(img, (28,28))
plt.imshow(img, cmap='gray')
plt.show()